```インポート```

In [23]:
import pandas as pd
from itertools import permutations

import gurobipy as gp
from gurobipy import GRB

import math
import sys
import time

```Input Data```

In [24]:
#車両台数5,避難所5,移動可能距離10km(1/14)

# 問題規模
M = int(1) #配送拠点数
S = int(5) #避難所数 max: 53
T = int(3)  #計画期間の長さ
Day=int(1)
#P = 1   #不明

# 計画期間
#PP = 3   #不明

# 車両
Cw = 3150 # 最大積載重量(kg)
Cv = 10 # 最大積載体積(pallet)


#車両が1度の配送で訪れることのできる被災拠点の箇所はCkカ所
Ckmax=2

#物資集積所が所有する車両数　#ここはM増やしたら変更
V=5#エラー対策で実数値代入形式

#物資情報のインポート→1単位あたりの重量w_i,1パレット辺りの個数,各ペナルティ係数c1_i~c3_i
supplies=["Drinking Water","Food","Blanket"]
#W = 3 #緊急物資の種類数
weight_perp_dict={
    "Drinking Water": 12,
    "Food": 1.2,
    "Blanket": 10
} ###1パレットあたりの重量(g計算)

numberofpieces_perp_dict={
    "Drinking Water": 80,
    "Food": 150,
    "Blanket": 36
} ###1パレットあたりの体積(立方メートルの値をとりあえず置換)

Penalty_coefficient1_dict = {
    "Drinking Water": 1.0,
    "Food": 1.0,
    "Blanket": 1.0
}

Penalty_coefficient2_dict = {
    "Drinking Water": 0.1,
    "Food": 0.1,
    "Blanket": 0.1
}

Penalty_coefficient3_dict = {
    "Drinking Water": 10,
    "Food": 10,
    "Blanket": 10
}

```データのインポート```

In [25]:
###② データのインポート###
# 避難所位置→美原区の避難所位置
dfa = pd.read_excel('Location_of_evacuation_shelters.xlsx',sheet_name="美原区避難所")
Shelters_location_tuples = gp.tuplelist(dfa.itertuples(index=False, name=None))
#SNumber=len(Shelters_name)
Shelters_name, Sx_coords, Sy_coords,Shelters_population = gp.multidict({
    row[0]: (row[1], row[2], row[3]) for row in Shelters_location_tuples
})

#避難所数が初期設定と違うときにエラーを示す
if S != len(Shelters_name):
    print("避難所数Sの初期設定の値を"+str(len(Shelters_name))+"に変更をしてください")
    sys.exit()

#物資集積所位置→美原区の物資集積所の位置
dfb = pd.read_excel('Location_of_evacuation_shelters.xlsx',sheet_name="美原区物資集積所")
MaterialDepots_location_tuples = gp.tuplelist(dfb.itertuples(index=False, name=None))
    #あとで消す
        #MaterialDepots_name=[row[0] for row in MaterialDepots_location_tuples]
        #MNumber=len(MaterialDepots_name)
MaterialDepots_name, Mx_coords, My_coords = gp.multidict({
    row[0]: (row[1], row[2]) for row in MaterialDepots_location_tuples
})

#物資集積所の数が初期設定と違うときにエラーを示す
if M != len(MaterialDepots_name):
    print("物資集積所Mの初期設定の値を"+str(len(MaterialDepots_name))+"に変更をしてください")
    sys.exit()

    
Shelters_and_MaterialDepots_name=Shelters_name+MaterialDepots_name #追加制約2で利用

```インポートしたデータの整理```

In [26]:
##③ インポートデータの整理→インポートデータから作成する辞書リストの作成##

def Distance_calculation(x1,y1,x2,y2):
    # 地球の半径（km）
    R = 6371.0

    # 度をラジアンに変換
    lat1_rad = math.radians(x1)
    lon1_rad = math.radians(y1)
    lat2_rad = math.radians(x2)
    lon2_rad = math.radians(y2)

    # 緯度の差と経度の差をラジアンで計算
    delta_lat = abs(lat2_rad - lat1_rad)
    delta_lon = abs(lon2_rad - lon1_rad)

    # 南北方向の距離（緯度方向）
    distance_ns = R * delta_lat

    # 東西方向の距離（経度方向）、緯度によってスケール補正
    distance_ew = R * delta_lon * math.cos((lat1_rad + lat2_rad) / 2)

    # マンハッタン距離を計算（南北方向 + 東西方向）
    distance = (distance_ns + distance_ew) * 1000  # メートルに変換
    return distance


#避難所間の辞書
distance_SSnode_dict={
    (s1, s2): Distance_calculation(Sx_coords[s1], Sy_coords[s1], Sx_coords[s2],Sy_coords[s2])
    for s1 in Shelters_name for s2 in Shelters_name if s1 != s2
}
CombinationSSnode,SSdistance=gp.multidict(distance_SSnode_dict)


#集積拠点→避難所の辞書
distance_MSnode_dict={
    (m1, s1): Distance_calculation(Mx_coords[m1], My_coords[m1], Sx_coords[s1],Sy_coords[s1])
    for m1 in MaterialDepots_name for s1 in Shelters_name if m1 != s1
}
distance_SMnode_update_dict={
    (s1, m1): Distance_calculation(Sx_coords[s1], Sy_coords[s1], Mx_coords[m1],My_coords[m1])
    for m1 in MaterialDepots_name for s1 in Shelters_name if m1 != s1
}
distance_MSnode_dict.update(distance_SMnode_update_dict)


CombinationMSnode,MSdistance=gp.multidict(distance_MSnode_dict)


#全ルート組み合わせの辞書
distance_Allnode_dict = {**distance_SSnode_dict, **distance_MSnode_dict}
CombinationAllnode,Alldistance=gp.multidict(distance_Allnode_dict)

#避難所kの物資iの総需要TD_ik
Shelters_DayilyDemand_dict = {
    s1: (
        float(Shelters_population[s1]) * T * 1.2,  # 物資1の需要
        float(Shelters_population[s1]) * T * 1.2,  # 物資2の需要
        int(float(Shelters_population[s1]) * 2)          # 物資3の需要
    )
    for s1 in Shelters_name
}
Shelters_DayilyDemand_name, Shelters_DayilyDemand1, Shelters_DayilyDemand2, Shelters_DayilyDemand3= gp.multidict(Shelters_DayilyDemand_dict)

#D_ikの数値の置き方考える #一旦割るTで考える ####################あとで考える要素
Shelters_Demand_dict = {
    s1: (
        float(Shelters_population[s1]) * 1.2,  # 物資1の需要
        float(Shelters_population[s1]) * 1.2,  # 物資2の需要
        int(float(Shelters_population[s1]) * 2/3)          # 物資3の需要
    )
    for s1 in Shelters_name
}
Shelters_Demand_name, Shelters_Demand1, Shelters_Demand2, Shelters_Demand3= gp.multidict(Shelters_Demand_dict)

#16式準備 #代入のやり方は確認しても良い
# i, j, 0 は初期条件
####################################################################12/10 14:55 初期条件の値の計算の続きから
total_demand = {
    "Drinking Water": sum(Shelters_DayilyDemand1[s] for s in Shelters_DayilyDemand_name),  # 物資1
    "Food": sum(Shelters_DayilyDemand2[s] for s in Shelters_DayilyDemand_name),   # 物資2
    "Blanket": sum(Shelters_DayilyDemand3[s] for s in Shelters_DayilyDemand_name)    # 物資3
}

# mp_dictの値を総需要量の50%に設定
mp_dict = {
    "Drinking Water": total_demand["Drinking Water"] * 0.5,
    "Food": total_demand["Food"] * 0.5,
    "Blanket": total_demand["Blanket"] * 0.5
}
mp_supplies,mp_amount=gp.multidict(mp_dict)

#避難所ごとの初期在庫量 #需要量の20%分の物資が存在
p_ik_dict={
    s1: (
        float(Shelters_population[s1]) * 1.2*0.2,  # 物資1の需要
        float(Shelters_population[s1]) * 1.2*0.2,  # 物資2の需要
        int(float(Shelters_population[s1]) * 2/3 *0.2)          # 物資3の需要
    )
    for s1 in Shelters_name
}

```Σ計算を実行するためのtappleリスト作成```

In [27]:
##④ 前処理→シグマ計算の変数の数を決めるtuppleリストの作成###
#避難所kごと
sheletrs=gp.tuplelist(Shelters_name)

#避難所kごとかつ物資iごと
list_variable_ik=[]
for k in Shelters_name:
    for i in supplies:
        tp=i,k
        list_variable_ik.append(tp)
variable_ik=gp.tuplelist(list_variable_ik)

#sl_jmt用
list_variable_jmt=[]
for j in MaterialDepots_name:
    for m in range(V):
        for t in range(T):
            tp=j,m,t+1
            list_variable_jmt.append(tp)
variable_jmt=gp.tuplelist(list_variable_jmt)



#x_abjmt用
list_variable_abjmt=[]
for a,b in CombinationAllnode:
    for j,m,t in variable_jmt:
        tp=a,b,j,m,t
        list_variable_abjmt.append(tp)
variable_abjmt=gp.tuplelist(list_variable_abjmt)

#Q_ijkmt用
list_variable_ijkmt=[]
for i,k in variable_ik:
    for j,m,t in variable_jmt:
        tp=i,j,k,m,t
        list_variable_ijkmt.append(tp)
variable_ijkmt=gp.tuplelist(list_variable_ijkmt)

#時刻tの配送拠点j
list_variable_jt=[]
for j in MaterialDepots_name:
    for t in range(T+1):
        tp=j,t+1
        list_variable_jt.append(tp)
variable_jt=gp.tuplelist(list_variable_jt)

#時刻tの配送拠点jの物資iの期末在庫量I_ijt
#t+1で数式がt=0~3になるように調整

list_variable_ijt=[]
        
for j in MaterialDepots_name:
    for i in supplies:
        for t in range(T+1):
            tp=i,j,t
            list_variable_ijt.append(tp)


variable_ijt=gp.tuplelist(list_variable_ijt)


##⑤ モデルの展開→変数の値を格納できるようaddVarを作成##
model = gp.Model('SuppliesDistributionPlan1') #モデルの定義

#時刻tに配送拠点jから避難所kに車両mが配送する物資iの配送量Q
# Q_ijkmt を整数変数として定義
Q_ijkmt = model.addVars(
    variable_ijkmt,
    vtype=GRB.INTEGER,  # 整数型として定義
    name="Q_ijkmt"
)
Q_numberofpieces_ijkmt= model.addVars(
    variable_ijkmt,
    vtype=GRB.INTEGER,  # 整数型として定義
    name="Q_ijkmt"
)


#時刻tに配送拠点Vを出発する車両数V_jt
V_jt=model.addVars(variable_jt,name="V_jt")

#x_abjmt
x_abjmt=model.addVars(
    variable_abjmt,
    vtype=gp.GRB.BINARY,  # バイナリ変数に設定
    name="x_abjmt"
)

#x_it+
x_ik_plus=model.addVars(variable_ik,name="x_ik_plus")

#x_it-
x_ik_minus=model.addVars(variable_ik,name="x_ik_minus")

#zx_ik
zx_ik=model.addVars(variable_ik,name="zx_ik")

#y_ik
y_ik=model.addVars(variable_ik,name="y_ik")

#n_ik
n_ik=model.addVars(variable_ik,name="n_ik")

#q_k
q_k = model.addVars(Shelters_name, name="q_k")
#最大供給率p
p=model.addVar(name="p")

#I_ijt
I_ijt=model.addVars(variable_ijt,name="I_ijt")

#sl_jmT   
sl_jmt = model.addVars(
    variable_jmt,  # j, m, t+1 のタプルリスト
    vtype=gp.GRB.BINARY,  # バイナリ変数に設定
    name="sl_jmt"
)

#slt_jmT→追加制約用   
slt_jmt = model.addVars(
    variable_jmt,  # j, m, t+1 のタプルリスト
    vtype=gp.GRB.BINARY,  # バイナリ変数に設定
    name="slt_jmt"
)

```制約式の記述```

In [28]:
##⑧ 制約式の記述##

#2式相当 
#################################################前期分不足量の値n_ikの代入してない

for k in Shelters_name:
    for i in supplies:
         model.addConstr(
            Shelters_DayilyDemand_dict[k][supplies.index(i)]
             *(x_ik_plus[i, k]-x_ik_minus[i, k])
             ==Shelters_Demand_dict[k][supplies.index(i)]-p_ik_dict[k][supplies.index(i)]- gp.quicksum(
                    Q_ijkmt[i, j, k, m, t+1] 
                    for j in MaterialDepots_name  
                    for m in range(V)
                    for t in range(T)
                ) ,
            name=f"x_ik_plusminus_constraint_{i}_{k}"
        )

#3式相当
BigNumber=10000000 ##############追加制約8でも使用
for k in Shelters_name:
    for i in supplies:
        model.addConstr(
            x_ik_plus[i,k] <= BigNumber*(1-zx_ik[i,k]),
            name=f"x_ik_plus_constraint_{i}_{k}"
        )

#4式相当
for k in Shelters_name:
    for i in supplies:
        model.addConstr(
            x_ik_minus[i,k] <= BigNumber*zx_ik[i,k],
            name=f"x_ik_minus_constraint_{i}_{k}"
        )

#######################################################################12/3 19:30 残り16式
#6,7式相当

max_q_k = model.addVars(Shelters_name, name="max_q_k") #6式の値をループ時に格納する変数
max_p = {k: model.addVar(name=f"max_p_{k}") for k in Shelters_name} #7式の値をループ時に格納する変数

for k in Shelters_name:
    for i in supplies:
        model.addConstr(
            max_q_k[k] >= Penalty_coefficient1_dict[i] * x_ik_plus[i, k],
            name=f"max_q_k_constraint_{i}_{k}"
        )
    # q_k に最大値を代入
    model.addConstr(q_k[k] == max_q_k[k], name=f"q_k_constraint_{k}")
    
    model.addConstr(
        max_p[k] >= Penalty_coefficient1_dict[i] * x_ik_plus[i, k],
        name=f"max_p_constraint_{k}"
    )
    # p に最大値を代入
    model.addConstr(p == max_p[k], name=f"q_k_constraint_{k}")


#8式相当
for k in Shelters_name:
    for i in supplies:
        Q_jmt_storage=gp.quicksum(
            Q_ijkmt[i, j, k, m, t+1]   # 各物資の重量を考慮
            for j in MaterialDepots_name  
            for m in range(V)
            for t in range(T)
            ) 

        # Shelters_DayilyDemand_dict から対応する物資の需要を取得
        daily_demand_storage = Shelters_DayilyDemand_dict[k][supplies.index(i)]  # iに対応する需要を取得
        
        # n_ik[i, k] - Q_jmt_storage が正の値になるように制約を先に行う
        #実質の非負制約
        model.addConstr(
            n_ik[i, k] - Q_jmt_storage >= 0,  
            name=f"y_ik_non_negative_constraint_{i}_{k}"
        )
        
        # 条件に基づいてy_ikの値を設定
        model.addConstr(
            y_ik[i, k] == (n_ik[i, k] - Q_jmt_storage) / daily_demand_storage,
            name=f"y_ik_constraint_{i}_{k}"
        )

#11式のみ考慮→palet数の整数条件から
for j in MaterialDepots_name:
    for m in range(V):  # V はトラックの数
        for t in range(T):  # T は計画期間
            for i in supplies:  # 物資の種類
                for k in Shelters_name:  # 避難所の数
                    model.addConstr(
                        Q_numberofpieces_ijkmt[i,j,k,m,t+1]==Q_ijkmt[i, j, k, m, t+1] / numberofpieces_perp_dict[i],
                        name=f"NumberOfPiecesIntegerConstraint_{i}_{j}_{k}_{m}_{t+1}"
                    )


#12、13式相当
for j in MaterialDepots_name:
    for m in range(V):  # V はトラックの数
        for t in range(T):  # T は計画期間
            model.addConstr(
                gp.quicksum(
                    Q_ijkmt[i, j, k, m, t+1] * weight_perp_dict[i]  # 各物資の重量を考慮
                    for i in supplies  # 物資の種類
                    for k in Shelters_name  # 避難所の数
                ) <= Cw,
                name=f"WeightConstraint_{j}_{m}_{t+1}"
            )
            

for j in MaterialDepots_name:
    for m in range(V):  # V はトラックの数
        for t in range(T):  # T は計画期間
            model.addConstr(
                gp.quicksum(
                    Q_numberofpieces_ijkmt[i,j,k,m,t+1]  
                    for i in supplies  # 物資の種類
                    for k in Shelters_name  # 避難所の数
                ) <= Cv,
                name=f"NumberOfPiecesConstraint_{j}_{m}_{t+1}"
            )

#16,17,18式相当
    #注意 r_ijtをまだ追加していない。必要であれば、model実装確認後に代入する
for j in MaterialDepots_name:
    for i in supplies:
        for t in range(T):

            #17式,18式準備
            I_ijt_StockAmount_constraint_storage= gp.quicksum(
                    Q_ijkmt[i, j, k, m, t+1]    # 各物資の重量を考慮
                    for m in range(V)  
                    for k in Shelters_name  # 避難所の数
            )

            #16式相当
            if t==0:
                model.addConstr(
                    I_ijt[i,j,t] == mp_amount[i], 
                    name=f"I_ijt_StockAmount_InitialConditions_{i}_{j}_{t}"
                )
                
            #17式相当
            model.addConstr(
                I_ijt_StockAmount_constraint_storage<= I_ijt[i,j,t-1+1], 
                name=f"I_ijt_StockAmount_constraint_{i}_{j}_{t-1+1}"
            )

            #18式相当
            model.addConstr(
                I_ijt[i,j,t+1]==I_ijt[i,j,t-1+1]-I_ijt_StockAmount_constraint_storage,
                name=f"I_ijt_StockAmount_definition_{i}_{j}_{t+1}"
            )

#print(CombinationAllnode)

#追加制約1 車両が必ず配送拠点から出発する制約
#移動先→避難所の集合
for j in MaterialDepots_name:
    for t in range(T):
        model.addConstr(
            gp.quicksum(
                x_abjmt[j, b, j, m, t+1] 
                for m in range(V)  # V はトラックの数
                for b in Shelters_name  # 避難所の数
            )== V_jt[j,t+1], 
            name=f"V_jt_MDdeparting_constraints_{j}_{t}"
        )
            
#追加制約2 次の進行先が存在する制約
#移動先→避難所の集合+物資集積拠点の集合
for j in MaterialDepots_name:
    for m in range(V):  # V はトラックの数
        for t in range(T):
            for k in Shelters_name:  # 避難所の数
                model.addConstr(
                    gp.quicksum(
                        x_abjmt[a, k, j, m, t+1] 
                        for a in Shelters_and_MaterialDepots_name 
                        if a!=k
                    )== gp.quicksum(
                        x_abjmt[k, b, j, m, t+1] 
                        for b in Shelters_and_MaterialDepots_name  
                        if k!=b
                    ), 
                    name=f"x_abjmt_BeforeNext_constraints_{k}_{j}_{m}_{t+1}"
                )
        

#追加制約3 車両は必ず配送拠点jに戻る制約
for j in MaterialDepots_name:
    for t in range(T):
        model.addConstr(
            gp.quicksum(
                x_abjmt[a, j, j, m, t+1] 
                for m in range(V)# V はトラックの数
                for a in Shelters_name  # 避難所の数
            )== V_jt[j,t+1], 
            name=f"V_jt_MDreturn_constraints_{j}_{t}"
        )

#追加制約4 sl_jmtの条件→2式追加
x_timelimt=10000##########################(積み込み+積み下ろし)＝10min×12パレットと見て、配送可能時間が1時間とする。1時間×10km/hをmaxで出す(5だと徒歩になる)
for j in MaterialDepots_name:
    for m in range(V):  # V はトラックの数
        for t in range(T):
            sl_jmt_constraint_storage= gp.quicksum(
                    distance_Allnode_dict[(a,b)]*x_abjmt[a, b, j, m, t+1]    #距離l_abの計算 
                    for a, b in CombinationAllnode  
                )
            
            model.addConstr(
            sl_jmt[j, m, t+1] *x_timelimt <= sl_jmt_constraint_storage,  
            name=f"sl_jmt_lower_limit_constraint_{j}_{m}_{t}"
            )
            model.addConstr(
                sl_jmt_constraint_storage <=(sl_jmt[j, m, t+1] +1)*x_timelimt ,  
                name=f"sl_jmt_upper_limit_constraint_{j}_{m}_{t}"
            )
            
            

#追加制約5
    #5-2 配送ルートが一定距離を超えたときにバイナリ変数が1を超える制約
for j in MaterialDepots_name:
    for m in range(V):  # V はトラックの数
         model.addConstr(
            sl_jmt[j, m, T] == 0,  # t = T の場合はバイナリ変数を0に固定
            name=f"sl_jmT_constraint_{j}_{m}"
        )


#追加制約6
for j in MaterialDepots_name:
    for t in range(T):
        model.addConstr(
             V_jt[j,t+1+1]<=V-gp.quicksum(
                sl_jmt[j, m, t+1]  
                for m in range(V)# V はトラックの数
            ), 
            name=f"V_jt_Mdepartingcapacity_constraints_{j}_{t+1}"
        )

#追加制約7
for j in MaterialDepots_name:
    for t in range(T):
        for m in range(V):  # V はトラックの数
            model.addConstr(
             gp.quicksum(
                x_abjmt[a, b, j, m, t+1] 
                for a, b in CombinationAllnode
                if a!=b
             )<=Ckmax+1, 
            name=f"Ckmax_constraints_{j}_{t+1}_{m}"
        )

#追加制約8 V_jt下限値設定
for j in MaterialDepots_name:
    for t in range(T):
        for m in range(V):  # V はトラックの数
            model.addConstr(
             BigNumber*slt_jmt[j,m,t+1]>=gp.quicksum(
                x_abjmt[a, b, j, m, t+1] 
                for a, b in CombinationAllnode
                if a!=b
             ), 
            name=f"slt_jmt_lowerlimit_{j}_{m}_{t+1}"
            )

for j in MaterialDepots_name:
    for t in range(T):
        model.addConstr(
            V_jt[j,t+1]==gp.quicksum(
                slt_jmt[j,m,t+1]
                for m in range(V)
            )
        )
        


#追加制約9 配送物資制約
for j in MaterialDepots_name:
    for m in range(V):  # V はトラックの数
        for t in range(T):
            for k in Shelters_name:  # 避難所の数
                model.addConstr(
                    gp.quicksum(
                        x_abjmt[a, k, j, m, t+1] 
                        for a in Shelters_and_MaterialDepots_name 
                        if a!=k
                    )*BigNumber>= gp.quicksum(
                        Q_ijkmt[i, j, k, m, t+1] 
                        for i in supplies
                    ), 
                    name=f"Q_Deliverymaterial_constraints_Q_{j}_{k}_{m}_{t+1}"
                )

```モデルの目的関数```

In [29]:
##⑨ モデルの実行##
P1_Absolute_ranking=1000000000
P2_Absolute_ranking=10000
P3_Absolute_ranking=0.0001

model.setObjective(
    P1_Absolute_ranking*(p+gp.quicksum(q_k[k] for k in Shelters_name)/len(Shelters_name))
    +P2_Absolute_ranking*gp.quicksum(Penalty_coefficient1_dict[i]*x_ik_plus[i, k]
                                     +Penalty_coefficient2_dict[i]*x_ik_minus[i,k]
                                     +Penalty_coefficient3_dict[i]*y_ik[i, k]
                                     for i in supplies for k in Shelters_name)
    +P3_Absolute_ranking*gp.quicksum(distance_Allnode_dict[(a,b)]*x_abjmt[a, b, j, m, t+1]    # 距離l_ab 
                                    for a, b in CombinationAllnode  # 避難所の数
                                    for t in range(T))
    , gp.GRB.MINIMIZE)

```コールバック関数の設定```

In [30]:
# 直近の保存時間（初期値として開始時間をセット）
last_save_time = time.time()
best_obj_so_far = float("inf")  # 最良目的関数値の初期値

def my_callback(model, where):
    global last_save_time, best_obj_so_far  

    if where == GRB.Callback.MIPSOL:  # 新しい解が見つかった場合
        obj_best = model.cbGet(GRB.Callback.MIPSOL_OBJ)

        # より良い解を記録
        if obj_best < best_obj_so_far:
            best_obj_so_far = obj_best

        #10分経過した段階での最良解を保存
        if time.time() - last_save_time >= 60:  
            last_save_time = time.time()  # 時間リセット
            save_solution(model, best_obj_so_far)  
            print(f"New best solution found: {best_obj_so_far}, saving...")

            
            
def save_solution(model, obj_best):
    #1 最大供給不足率q_k
    q_k_data=[]
    for k, var in q_k.items() :
        q_k_data.append({
            "避難所": k,
            "避難所ごとの最大供給不足率q_k":model.cbGetSolution(var)
        })

    q_k_df = pd.DataFrame(q_k_data)
    # インデックスを空に設定
    q_k_df.index = [''] * len(q_k_df)
    
    

    #2 不足率・過剰率・前期不足率
    xy_ik_data=[]

    for (i, k), var in x_ik_plus.items():
        x_ik_plus_value = model.cbGetSolution(var)  # 不足率 x_ik_plus の値
        x_ik_minus_value = model.cbGetSolution(x_ik_minus[i, k])  # 過剰率 x_ik_minus の値
        y_ik_value = model.cbGetSolution(y_ik[i, k])  # 前期不足率 y_ik の値
        
        
        xy_ik_data.append({
                "避難所k": k,
                "物資i": i,
                "不足率x_ik_plus":x_ik_plus_value,
                "過剰率x_ik_minus": x_ik_minus_value,
                "前期不足率y_ik":y_ik_value
        })
    
    xy_ik_df = pd.DataFrame(xy_ik_data)

    sorted_xy_ik_df = xy_ik_df.sort_values(
        by=[ "避難所k", "物資i", "不足率x_ik_plus", "過剰率x_ik_minus", "前期不足率y_ik"],
        ascending=[True, True, True, True, True]
    )
    # インデックスを空に設定
    sorted_xy_ik_df.index = [''] * len(sorted_xy_ik_df)


    #3 移動距離と移動先
    # DataFrameを作成
    route_data=[]

    for (a, b, j, m, t), var in x_abjmt.items():
        route_data.append({
                "時刻t": t,
                "集積拠点j": j,
                "車両m": m,
                "移動元a": a,
                "移動先b": b,
                "移動距離":Alldistance.get((a, b), "N/A"),
                "x_abjmt":model.cbGetSolution(var) 
        })
    
    route_df = pd.DataFrame(route_data)

    sorted_route_df = route_df.sort_values(
        by=["時刻t", "集積拠点j", "車両m", "移動元a", "移動先b","移動距離", "x_abjmt"],
        ascending=[True, True, True, True, True,True,True]  # 全て昇順
    )
    # インデックスを空に設定
    sorted_route_df.index = [''] * len(sorted_route_df)
    

    #4 配送量
    Q_ijkmt_data=[]

    for (i, j, k, m, t), var in Q_ijkmt.items():
        
        Q_numberofpieces_value =  model.cbGetSolution(Q_numberofpieces_ijkmt[i, j, k, m, t])
        
        
        Q_ijkmt_data.append({
                "時刻t": t,
                "集積拠点j": j,
                "車両m": m,
                "避難所k": k,
                "物資i": i,
                "配送量Q":model.cbGetSolution(var),
                "パレット数Q_numberofpieces": Q_numberofpieces_value
        })
    
    Q_ijkmt_df = pd.DataFrame(Q_ijkmt_data)

    sorted_Q_ijkmt_df = Q_ijkmt_df.sort_values(
        by=["時刻t", "集積拠点j", "車両m", "避難所k", "物資i", "配送量Q", "パレット数Q_numberofpieces"],
        ascending=[True, True, True, True, True, True, True]
    )
    # インデックスを空に設定
    sorted_Q_ijkmt_df.index = [''] * len(sorted_Q_ijkmt_df)


    #5 
    V_jt_data=[]

    for ( j, t), var in V_jt.items():
        V_jt_data.append({
                "時刻t": t,
                "集積拠点j": j,
                "稼働車両数V_jt":model.cbGetSolution(var)
        })
    
    V_jt_df = pd.DataFrame(V_jt_data)

    sorted_V_jt_df = V_jt_df.sort_values(
        by=["時刻t", "集積拠点j", "稼働車両数V_jt"],
        ascending=[True, True, True]  # 全て昇順
    )
    # インデックスを空に設定
    sorted_V_jt_df.index = [''] * len(sorted_V_jt_df)



    # Excel に保存
    with pd.ExcelWriter("best_solution.xlsx", engine='openpyxl') as writer:

        q_k_df.to_excel(writer, sheet_name="q_k_data", index=False)

        sorted_xy_ik_df.to_excel(writer, sheet_name="xy_ik_data", index=False)

        sorted_route_df.to_excel(writer, sheet_name="x_abjmt_data", index=False)

        sorted_Q_ijkmt_df.to_excel(writer, sheet_name="Q_ijkmt_data", index=False)

        sorted_V_jt_df.to_excel(writer, sheet_name="V_jt_data", index=False)
    

    
    

```モデルの実行```

In [31]:
model.write('SuppliesDistributionPlan1.rlp')

In [32]:
model.setParam(GRB.Param.TimeLimit, 600)  # 4日間（345600秒）の制限
model.optimize(my_callback)

Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 608 rows, 1037 columns and 5642 nonzeros
Model fingerprint: 0x6910e208
Variable types: 107 continuous, 930 integer (480 binary)
Coefficient statistics:
  Matrix range     [7e-05, 1e+07]
  Objective range  [7e-02, 1e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+07]
Presolve removed 318 rows and 300 columns
Presolve time: 0.01s
Presolved: 290 rows, 737 columns, 3940 nonzeros
Variable types: 34 continuous, 703 integer (475 binary)
Found heuristic solution: objective 5.335817e+08
Found heuristic solution: objective 5.335817e+08

Root relaxation: objective 3.619815e+08, 452 iterations, 0.00 seconds (0.00 work units)

    Nodes    

```結果出力```

In [33]:
##⑩ 結果出力

print("\n\n -----------------------------------------------------------")
print("\n\n -----------------------------------------------------------")

#print("q_k[k]の値")
print("_________________________________________________________________________________")
print("\n 目的関数第一項について")
print("pの値：%f",p.X)

print("_________________________________________________________________________________\n")

# Gurobiモデルが最適化されていることを確認
if model.Status == 9 :  # GRB.OPTIMAL の場合
    # DataFrameを作成はスキップでOK→q_kしかデータがないので
    q_k_data=[]
    for k, var in q_k.items() :
        q_k_data.append({
            "避難所": k,
            "避難所ごとの最大供給不足率q_k":var.X
        })

    q_k_df = pd.DataFrame(q_k_data)
    # インデックスを空に設定
    q_k_df.index = [''] * len(q_k_df)

    # 表示
    print(q_k_df)
else:
    print("モデルが最適化されていません。最適化を実行してください。")




print("\n\n _________________________________________________________________________________")
print("\n 目的関数第二項について")
if model.Status == 2 or 9:  # GRB.OPTIMAL の場合
    # DataFrameを作成
    xy_ik_data=[]

    for (i, k), var in x_ik_plus.items():
       # if var.X !=0:  # 値が0でない値のみ出力
        
        x_ik_minus_value = (
                x_ik_minus[i, k].X 
        )
        
        y_ik_value = (
                y_ik[i,k].X
        )
        
        
        xy_ik_data.append({
                "避難所k": k,
                "物資i": i,
                "不足率x_ik_plus":var.X,
                "過剰率x_ik_minus": x_ik_minus_value,
                "前期不足率y_ik":y_ik_value
        })
    
    xy_ik_df = pd.DataFrame(xy_ik_data)

    sorted_xy_ik_df = xy_ik_df.sort_values(
        by=[ "避難所k", "物資i", "不足率x_ik_plus", "過剰率x_ik_minus", "前期不足率y_ik"],
        ascending=[True, True, True, True, True]
    )
    # インデックスを空に設定
    sorted_xy_ik_df.index = [''] * len(sorted_xy_ik_df)

    # 表示
    print(sorted_xy_ik_df)
else:
    print("モデルが最適化されていません。最適化を実行してください。")

print("\n\n _________________________________________________________________________________")
print("\n 目的関数第三項について")

if model.Status == 2 or 9:  # GRB.OPTIMAL の場合
    # DataFrameを作成
    route_data=[]

    for (a, b, j, m, t), var in x_abjmt.items():
       # if var.X ==1:  # 値が1の場合だけ考慮
        route_data.append({
                "時刻t": t,
                "集積拠点j": j,
                "車両m": m,
                "移動元a": a,
                "移動先b": b,
                "移動距離":Alldistance.get((a, b), "N/A"),
                "x_abjmt":var.X 
         })
    
    route_df = pd.DataFrame(route_data)

    sorted_route_df = route_df.sort_values(
        by=["時刻t", "集積拠点j", "車両m", "移動元a", "移動先b","移動距離", "x_abjmt"],
        ascending=[True, True, True, True, True,True,True]  # 全て昇順
    )
    # インデックスを空に設定
    sorted_route_df.index = [''] * len(sorted_route_df)

    # 表示
    print(sorted_route_df)
    
else:
    print("モデルが最適化されていません。最適化を実行してください。")



print("\n\n _________________________________________________________________________________")
print("\n 結果の詳細確認その1 ：　物資配送量Q_ijkmt")
if model.Status == 2 or 9:  # GRB.OPTIMAL の場合
    # DataFrameを作成
    Q_ijkmt_data=[]

    for (i, j, k, m, t), var in Q_ijkmt.items():
        
        Q_numberofpieces_value = (
                Q_numberofpieces_ijkmt[i, j, k, m, t].X #if (i, j, k, m, t) in Q_numberofpieces_ijkmt else None
        )
        
        Q_ijkmt_data.append({
                "時刻t": t,
                "集積拠点j": j,
                "車両m": m,
                "避難所k": k,
                "物資i": i,
                "配送量Q":var.X,
                "パレット数Q_numberofpieces": Q_numberofpieces_value
        })
    
    Q_ijkmt_df = pd.DataFrame(Q_ijkmt_data)

    sorted_Q_ijkmt_df = Q_ijkmt_df.sort_values(
        by=["時刻t", "集積拠点j", "車両m", "避難所k", "物資i", "配送量Q", "パレット数Q_numberofpieces"],
        ascending=[True, True, True, True, True, True, True]
    )
    # インデックスを空に設定
    sorted_Q_ijkmt_df.index = [''] * len(sorted_Q_ijkmt_df)

    # 表示
    print(sorted_Q_ijkmt_df)
else:
    print("モデルが最適化されていません。最適化を実行してください。")

print("\n\n _________________________________________________________________________________")
print("\n 結果の詳細確認その2 ：　V_jt")
if model.Status == 2 or 9:  # GRB.OPTIMAL の場合
    # DataFrameを作成
    V_jt_data=[]

    for ( j,  t), var in V_jt.items():
         V_jt_data.append({
                "時刻t": t,
                "集積拠点j": j,
                "稼働車両数V_jt":var.X
        })
    
    V_jt_df = pd.DataFrame(V_jt_data)

    sorted_V_jt_df = V_jt_df.sort_values(
        by=["時刻t", "集積拠点j", "稼働車両数V_jt"],
        ascending=[True, True, True]  # 全て昇順
    )
    # インデックスを空に設定
    sorted_V_jt_df.index = [''] * len(sorted_V_jt_df)

    
else:
    print("モデルが最適化されていません。最適化を実行してください。")





#まとめて表示
with pd.ExcelWriter("final_solution.xlsx", engine='openpyxl') as writer:

    q_k_df.to_excel(writer, sheet_name="q_k_data", index=False)
    
    sorted_xy_ik_df.to_excel(writer, sheet_name="xy_ik_data", index=False)
    
    sorted_route_df.to_excel(writer, sheet_name="x_abjmt_data", index=False)

    sorted_Q_ijkmt_df.to_excel(writer, sheet_name="Q_ijkmt_data", index=False)

    sorted_V_jt_df.to_excel(writer, sheet_name="V_jt_data", index=False)




 -----------------------------------------------------------


 -----------------------------------------------------------
_________________________________________________________________________________

 目的関数第一項について
pの値：%f 0.13311176969552693
_________________________________________________________________________________

 避難所  避難所ごとの最大供給不足率q_k
  sa          0.266667
  sb          0.266667
  sc          0.266667
  se          0.131782
  sf          0.252548


 _________________________________________________________________________________

 目的関数第二項について
 避難所k             物資i  不足率x_ik_plus  過剰率x_ik_minus  前期不足率y_ik
   sa         Blanket      0.132546            0.0        0.0
   sa  Drinking Water      0.266667            0.0        0.0
   sa            Food      0.266667            0.0        0.0
   sb         Blanket      0.133113            0.0        0.0
   sb  Drinking Water      0.266667            0.0        0.0
   sb            Food      0.266667            0.0        0